# Asymptotics of Algebraic Generating Functions
This is a companion notebook for Chapter 4 of the thesis, modified from the upcoming paper, Asymptotics of multivariate algebraic generating functions, by Torin Greenwood, Stephen Melczer, Tiadora Ruza and Mark Wilson. 

This notebook provides functions to allow one to compute a embeddings of an algebraic generating function as the diagonal of a rational generating function using either Denef & Lipshitz embedding method or Safonov's algorithm. As well, this notebook provides code related to the examples found in Chapter 4 of the thesis. These serve both as a companion and as informative results.

In [185]:
# Define variables and parameters
var('x', 'y', 'z', 'Y')
var('p1', 'q2', 'p3', 'q1', 'q2', 'p2', 'q3')

(p1, q2, p3, q1, q2, p2, q3)

## Code for Denef & Lipshitz Method

In [186]:
def embed_algebraic_DL(P, Q, W, Y=None, params=[]):
    r"""Function to embed a d-variable algebraic series f(x) as a diagonal of a (d+1)-variable
    rational function R(x,Y). Requires specific input whose existence is guaranteed by results of
    Denef and Lipshitz.

    INPUT:

    * ``P``      -- Polynomial in QQ[x,Y] such that P(x,f(x)) = 0.
    * ``Q``      -- Polynomial in QQ[x,Y] such that Q(0,0) = 0 and Q_Y(0,0) != 0.
    * ``W``      -- Rational function in QQ(x,Y) such that W(x,phi(x)) = f(x) for a power series phi(x)
                    vanishing at the origin such that Q(x,phi(x))=0.
    * ``Y``      -- (Optional) The variable of P(x,Y) for which Y = f(x) is a root. By default, 
                    the final variable returned by P.variables().
    * ``params`` -- (Optional) List of symbolic variables appearing in P that are considered parameters, 
                    meaning they remain after coefficient extraction.

    OUTPUT:

    A (d+1)-variable rational function R(x,Y) such that [x^r]f(x) = [x^rY^|r|]R(x,Y) for all r in N^d,
    where |r| = r_1 + ... + r_d. Returns error if P, Q, and W do not satisfy the required conditions.

    NOTE: The code verifies that W(x,phi(x)) = r(x) for some r with P(x,r(x)) = 0 but does not verify
    that r(x) is the specific root f(x) of P. An optional argument to check this will be added later.
    """

    ########################################################
    # Set variables and verify assumptions on P, Q, and W
    ########################################################
    P_variables = [v for v in P.variables() if not v in params]
    Q_variables = [v for v in Q.variables() if not v in params]
    W_variables = [v for v in W.variables() if not v in params]

    if len(P_variables) == 0:
        raise ValueError("No non-parameter variables of P detected.")
        
    if not (set(Q_variables) | set(W_variables)).issubset(set(P_variables)):
        raise ValueError("Q and W have variables not found in P.")
    
    if Y == None:
        local_Y = P_variables[-1]
    else:
        local_Y = Y

    if  local_Y not in P_variables:
        raise ValueError("Y is not a (non-parameter) variable of P.")

    sub = [k==0 for k in P_variables]
    if Q.subs(sub).expand() != 0:
        raise ValueError("Q does not vanish at the origin.")
    if Q.diff(local_Y).subs(sub).expand() == 0:
        raise ValueError("Q_Y vanishes at the origin.")

    if (P.subs(local_Y==W).numerator()).resultant(Q, local_Y) != 0:
        raise ValueError("Y = W(x,phi(x)) is not a root of P(x,Y).")

    ########################################################
    # Compute and return rational function R(x,Y)
    ########################################################
    R = local_Y * diff(Q,local_Y) * W / Q
    R = R.subs({v:v*local_Y for v in P_variables if not v == local_Y})
    return(R.factor())

## Code for Safonov's Algorithm

In [187]:
def lowest_degree_terms(pol, params=[]):
    """
    Find the order of vanishing and terms with lowest degree in a multivariate polynomial.

    INPUT:

    * ``pol``    -- A polynomial.
    * ``params`` -- (Optional) List of symbolic variables appearing in pol that are considered parameters.

    OUTPUT:

    A tuple containing the order of vanishing of pol and its terms with lowest degree.
    """
    pol_symbolic    = SR(str(pol))
    params_symbolic = [SR(v) for v in params]
    pol_variables   = [v for v in pol_symbolic.variables() if not v in params_symbolic]
    
    # If constant return the constant -- includes the case pol = 0
    if len(pol_variables) == 0:
        return 0, pol_symbolic

    # Otherwise 
    pol_power_series = PowerSeriesRing(PolynomialRing(QQ,names=params_symbolic), names=pol_variables)(pol_symbolic) 
    mu = pol_power_series.valuation()

    # Univariate power series return coefficients not monomials -- check which case we are in
    ps_variables = parent(pol_power_series).gens()
    if len(ps_variables) == 1: 
        return mu, SR(pol_power_series[mu]*ps_variables[0]^mu)
    else:
        return mu, SR(pol_power_series[mu])

In [188]:
def lowest_residue_terms(P, terms, Y=None, params=[]):
    """
    Find the order of vanishing and terms with lowest degree in a multivariate polynomial.

    INPUT:

    * ``P``        -- Polynomial P(Y,x) where P(Y,x) = (Y-u(x))G(Y,x) with u(x) 
                      a simple root of P (meaning G(u(x),x) not identically zero) 
    * ``terms``    -- A function that takes a natural number n and returns the series terms of u(x) up to order n
    * ``Y``        -- (Optional) The variable of P(x,Y) for which Y = u(x) is a root. By default, 
                      the final variable returned by P.variables().
    * ``params``   -- (Optional) List of symbolic variables appearing in P that are considered parameters, 
                      meaning they remain after coefficient extraction.
    * ``Safonov``  -- (Optional) Boolean that, if True, also returns the expansion of G(u(x),x) to order q = mu(nu+1)/2
                      to be used in the Safonov embedding algorithm.

    OUTPUT:

    The degree mu to which G(u(x),x) vanishes at the origin and the sum of terms in G(u(x),x) of degree mu.
    """
    # Define variables and ring
    P_variables = [v for v in P.variables() if v not in params]
    R = PolynomialRing(PolynomialRing(QQ,names=params).fraction_field(), names = P_variables)

    if len(P_variables) == 0:
        raise ValueError("No non-parameter variables of P detected.")
    
    if Y == None:
        local_Y = P_variables[-1]
    else:
        local_Y = Y

    if  local_Y not in P_variables:
        raise ValueError("Y is not a (non-parameter) variable of P.")

    P_x_variables = [v for v in P_variables if not v == local_Y]

    P_derivative = P.diff(local_Y)
    N = 0
    G_series = P_derivative.subs(Y==terms(N)).taylor(P_x_variables,0,N)
    while G_series == 0:
        N = N + 1
        G_series = P_derivative.subs(Y==terms(N)).taylor(P_x_variables,0,N)
    return lowest_degree_terms(G_series)

In [189]:
def update_M_matrix(M, a, i, mu):
    """
    Update the matrix 'M' of linear maps and the vector 'a' of changes that occur with the Safonov embedding algorithm.

    INPUT:

    * ``M``   -- A (dxd) integer matrix.
    * ``a``   -- A length d integer vector.
    * ``i``   -- Integer between 1 and d representing the index of the variable updated in Safonov's algorithm.
    * ``mu``  -- The "order of vanishing" computed in Safonov's algorithm.

    OUTPUT:

    The matrix M and vector a updated to reflect the change of variable made in Safonov's algorithm.
    """
    d = M.dimensions()[0]
    M_matrix_update = matrix(ZZ, d, d)
    
    for k in [0..d-1]:
        for j in [0..d-1]:
            if ((j == i) | (k == j)):
                M_matrix_update[k,j] = 1
            else:
                M_matrix_update[k,j] = 0
                
    new_a = a*M_matrix_update
    new_a[0,i] = new_a[0,i] + mu
    return M*M_matrix_update, new_a

In [190]:
def embed_algebraic_Safonov(P, terms, Y=None, params=[]):
    r"""Function to embed a d-variable algebraic series f(x) as an "M-diagonal" of a (d+1)-variable
    rational function R(x,Y) following an algorithm of Safonov.

    INPUT:

    * ``P``      -- Polynomial in QQ[x,Y] such that P(x,f(x)) = 0.
    * ``terms``  -- A function that takes a natural number n and returns the series terms of u(x) up to order n
    * ``Y``      -- (Optional) The variable of P(x,Y) for which Y = f(x) is a root. By default, 
                    the final variable returned by P.variables().
    * ``params`` -- (Optional) List of symbolic variables appearing in P that are considered parameters, 
                    meaning they remain after coefficient extraction.

    OUTPUT:

    A (d+1)-variable rational function R(x,Y) and (dxd) matrix M such that [x^r]f(x) = [x^rY^|Mr|]R(x,Y) 
    for all r in N^d, where |r| = r_1 + ... + r_d. 
    """

    ########################################################
    # Set variables and perform basic setup
    ########################################################
    P_variables = [v for v in P.variables() if not v in params]

    if len(P_variables) == 0:
        raise ValueError("No non-parameter variables of P detected.")
            
    if Y == None:
        local_Y = P_variables[-1]
    else:
        local_Y = Y

    if  local_Y not in P_variables:
        raise ValueError("Y is not a (non-parameter) variable of P.")

    P_x_variables = [v for v in P_variables if not v == local_Y]
    P_variables = P_x_variables + [local_Y]
    
    P_square_free = prod([f for (f,_) in P.factor_list()])
    P_derivative = P_square_free.diff(local_Y)

    ########################################################
    # Run main algorithm
    ########################################################
    # Write P(Y,x) = (y-f(x))G(Y,x) where G(f(x),x) is not identically zero
    # Compute order of vanishing and lowest order terms of G(f(x),x)
    mu, lowest_terms = lowest_residue_terms(P, terms, Y=local_Y, params=params)
    q = (mu*(mu+1))/2

    # Compute f(x) to appropriate accuracy then make a substition into P
    Sq = terms(q) 
    P_square_free = P_square_free.subs(Y == Y + Sq)
    P_derivative = P_derivative.subs(Y == Y + Sq)

    # Define matrix M and vector a
    d = len(P_x_variables)
    M = matrix.identity(d)
    a = matrix(1, d)

    R_ring = PolynomialRing(PolynomialRing(QQ, names= params), names = P_x_variables)
    R_ring_Y = PolynomialRing(PolynomialRing(QQ, names= params), names = P_variables)
    lowest_terms_poly = R_ring(str(lowest_terms))


    while (mu != 0):
        # Find variable zeta of maximum degree in lowest_terms 
        max_degree = max([max(e) for e in lowest_terms_poly.exponents()])
        zeta = list(filter(lambda v: lowest_terms_poly.degree(R_ring(v)) == max_degree, P_x_variables))[0]

        # Define and make monomial substitution with zeta
        sbs = [Y == Y*zeta^mu] + [v == v*zeta for v in P_x_variables if str(v) != str(zeta)]
        P_square_free = P_square_free.subs(sbs)
        P_derivative = P_derivative.subs(sbs)
        Sq = SR(Sq).subs(sbs)

        # Update M matrix and a vector with substition
        M, a = update_M_matrix(M, a, P_x_variables.index(zeta), mu)

        # Factor out power of zeta
        P_derivative = (P_derivative/(zeta^mu)).simplify_full()       
        P_square_free_poly = R_ring_Y(P_square_free)
        zeta_valuation = min([m.degree(R_ring_Y(zeta)) for m in P_square_free_poly.monomials()])
        P_square_free = (P_square_free/(zeta^zeta_valuation)).simplify_full()
        
        # Find new value of mu
        mu, lowest_terms = lowest_degree_terms(P_derivative.subs(Y==0), params=[])
        lowest_terms_poly = R_ring(str(lowest_terms))

    # if required, subtract off constant
    sbs = [v == 0 for v in P_variables]
    if P_square_free.taylor(P_variables, 0, 0).subs(sbs) != 0:
        P_square_free = P_square_free - P_square_free.taylor(P_variables, 0, 0).subs(sbs)

    # Return embedding, matrix which gives diagonal correspondance and vector
    subs = [v == v*Y for v in P_x_variables]
    leading_product = prod([P_x_variables[i]^a[0,i] for i in range(d)])
    R = ((leading_product*((Y^2*diff(P_square_free,Y))/P_square_free)+Sq).subs(subs)).simplify_full()
    return R, M.transpose()

### Example 62.

In [191]:
f = x*sqrt(1-x-y)
P = Y^2 - x^2*(1-x-y)
Q = (1-Y)^2-(1-x-y)
W = x*(1-Y)

In [192]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' -2*(Y - 1)^2*Y*x/(Y + x + y - 2)

In [193]:
g = x/sqrt(1-x-y)
P = Y^2*(1-x-y) - x^2
Q = (1-Y)^2-(1-x-y)
W = x/(1-Y)

In [194]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' -2*Y*x/(Y + x + y - 2)

### Example 63.

In [195]:
f = (1-sqrt(1-4*x))/(2*x)
P = Y^2*x-Y+1
Q = Y^2*x+2*Y*x-Y+x
W = Y+1

In [196]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' (2*Y^2*x + 2*Y*x - 1)*(Y + 1)/(Y^2*x + 2*Y*x + x - 1)

In [197]:
Q = Y^2-2*x*Y+x^2+Y
W = -Y/x+1

In [198]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' -(2*Y*x - 2*Y - 1)*(x - 1)/((Y*x^2 - 2*Y*x + Y + 1)*x)

### Example 67. 0-2-5 Trees

In [199]:
P = 1-Y+y*(((Y-1)^2-1)+x*(Y-1)^5)
Q = -Y+y*(Y^2-1+x*Y^5)
W = Y+1

In [200]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' (5*Y^6*x*y + 2*Y^2*y - 1)*(Y + 1)/(Y^6*x*y + Y^2*y - y - 1)

### Example 68.

In [201]:
f = sqrt((1-y)/(1-x))
P = (1-x)*Y^2-(1-y)
show(-4*A*C)

4*(x - 1)*(y - 1)

In [202]:
C = y-1
A = 1-x

Dt = (x-1)*(y-1)
Q = (Y-1)^2 - Dt
W = Dt/(A*(1-Y))

In [203]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)

'Denef & Lipshitz Embedding: ' -2*(Y*y - 1)/(Y*x*y - Y - x - y + 2)

### Example 70. Narayana Generating Function

In [214]:
f = 1/(2*x)*(1-x*(y-1)-sqrt(1-2*x*(y+1)+x^2*(y-1)^2))-1
P = Y^2*x + Y*x*y + Y*x + x*y - Y
def terms(n): return f.taylor([x, y], 0, n)

In [216]:
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Safonov Rational Embedding: ", ratSaf.factor())
show("A: ", A)

'Safonov Rational Embedding: ' (Y^2*x*y + 2*Y^2*x + Y*x - 1)*Y/(Y^2*x*y + Y^2*x + Y*x*y + Y*x - 1)

'A: ' [1 0]
[0 1]

In [217]:
f = 1/2*(1-x*(y-1)-sqrt(1-2*x*(y+1)+x^2*(y-1)^2))
P = Y*x*y+Y^2-Y*x-Y+x
def terms(n): return f.taylor([x, y], 0, n)

In [218]:
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Safonov Rational Embedding: ", ratSaf.factor())
show("A: ", A)

'Safonov Rational Embedding: ' (Y^2*x*y - Y*x + 2*Y - 1)*Y/(Y^2*x*y - Y*x + Y + x - 1)

'A: ' [1 0]
[0 1]

### Example 71. Polygon Dissections

In [219]:
f = (4862*x^9 + 19448*x^7 + 8008*x^6 + 27027*x^5 + 16016*x^4 + 16302*x^3 + 7788*x^2 + 3058*x + 540)*y^11 + (1430*x^8 + 5005*x^6 + 2002*x^5 + 5720*x^4 + 3080*x^3 + 2475*x^2 + 890*x + 191)*y^10 + (429*x^7 + 1287*x^5 + 495*x^4 + 1155*x^3 + 540*x^2 + 309*x + 64)*y^9 + (132*x^6 + 330*x^4 + 120*x^3 + 216*x^2 + 80*x + 25)*y^8 + (42*x^5 + 84*x^3 + 28*x^2 + 35*x + 8)*y^7 + (14*x^4 + 21*x^2 + 6*x + 4)*y^6 + (5*x^3 + 5*x + 1)*y^5 + (2*x^2 + 1)*y^4 + x*y^3 + y^2
P = -x*Y^3 + x*Y^2*y - Y*y^3 + y^4 + Y^3 + Y^2*y - Y*y^2
def terms(n): return f.taylor([x, y],0,n)

In [220]:
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Safonov Rational Embedding: ", ratSaf.factor())
show("A: ", A)

'Safonov Rational Embedding: ' (3*Y^7*x*y^3 + 7*Y^6*x*y^3 + 6*Y^5*x*y^3 - 2*Y^5*x*y^2 + 3*Y^4*x*y^3 - 3*Y^5*y^2 - 3*Y^4*x*y^2 + Y^3*x*y^3 - 7*Y^4*y^2 - 2*Y^3*x*y^2 - 6*Y^3*y^2 - Y^2*x*y^2 - 2*Y^3*y - 3*Y^2*y^2 - 2*Y^2*y - Y*y^2 - Y*y + Y + 1)*Y^2*y^2/(Y^6*x*y^3 + 3*Y^5*x*y^3 + 3*Y^4*x*y^3 - Y^4*x*y^2 + Y^3*x*y^3 - Y^4*y^2 - 2*Y^3*x*y^2 - 3*Y^3*y^2 - Y^2*x*y^2 - 3*Y^2*y^2 - Y^2*y - Y*y^2 - Y*y + 1)

'A: ' [1 0]
[1 1]

### Example 73. Assembly Trees

In [118]:
f = 1 - sqrt((1 - x)^2 + (1 - y)^2 - 1)
P = (Y - 1)^2 - ((1 - x)^2 + (1 - y)^2 - 1)
def terms(n): return f.taylor([x, y], 0, n)

In [119]:
ratDL = embed_algebraic_DL(P, Q=P, W=Y, Y=Y)
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)
show("Safonov Rational Embedding: ", ratSaf)
show("A: ", A)

'Denef & Lipshitz Embedding: ' -2*(Y - 1)*Y/(Y*x^2 + Y*y^2 - Y - 2*x - 2*y + 2)

'Safonov Rational Embedding: ' -2*(Y^2 - Y)/(Y*x^2 + Y*y^2 - Y - 2*x - 2*y + 2)

'A: ' [1 0]
[0 1]

### Example 74. New and Old Leaves of Binary Trees

In [120]:
f = -(1/2)*(z*y+sqrt(y^2*z^2-4*x*z^2+2*y*z^2-2*y*z+z^2-2*z+1)-z-1)/z - 1
P = ((Y+1) - 1)*(1 - z*((Y+1) - 1 + y)) - z*((Y+1) - 1 + x)
def terms(n): return f.taylor([x, y, z], 0, n)

In [121]:
ratDL = embed_algebraic_DL(P, Q=P, W=Y, Y=Y)
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)
show("Safonov Rational Embedding: ", ratSaf)
show("A: ", A)

'Denef & Lipshitz Embedding: ' (Y^2*y*z + 2*Y^2*z + Y*z - 1)*Y/(Y^2*y*z + Y^2*z + Y*x*z + Y*z - 1)

'Safonov Rational Embedding: ' ((Y^3*y + 2*Y^3 + Y^2)*z - Y)/((Y^2*y + Y^2 + Y*x + Y)*z - 1)

'A: ' [1 0 0]
[0 1 0]
[0 0 1]

### Example 75. Stochastic Context-Free Grammars

In [122]:
f = ((1 - p1*q2*x)*(1 - p3*x^2*y) - sqrt((1 - p1*q2*x)^2*(1-p3*x^2*y)^2-4*p2*q1*q2*q3*x^3*y*(1-p3*x^2*y)))/(2*p2*q3*x^2*y)
P = Y*p1*p3*q2*y*x^3 - Y^2*p2*q3*y*x^2 + p3*q1*q2*y*x^3 - Y*p3*y*x^2 - Y*p1*q2*x - q1*q2*x + Y

def terms(n): return f2.taylor([x, y], 0, n)

In [123]:
ratDL = embed_algebraic_DL(P, Q=P, W=Y, Y=Y, params=[p1, p2, p3, q1, q2, q3])
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y, params=[p1, p2, p3, q1, q2, q3])
show("Denef & Lipshitz Embedding: ", ratDL)
show("Safonov Rational Embedding: ", ratSaf)
show("A: ", A)

'Denef & Lipshitz Embedding: ' (Y^4*p1*p3*q2*x^3*y - 2*Y^4*p2*q3*x^2*y - Y^3*p3*x^2*y - Y*p1*q2*x + 1)*Y/(Y^4*p1*p3*q2*x^3*y + Y^3*p3*q1*q2*x^3*y - Y^4*p2*q3*x^2*y - Y^3*p3*x^2*y - Y*p1*q2*x - q1*q2*x + 1)

'Safonov Rational Embedding: ' (Y^2*p1*q2*x - (Y^5*p1*p3*q2*x^3 - (2*Y^5*p2*q3 + Y^4*p3)*x^2)*y - Y)/((Y*p1 + q1)*q2*x - ((Y^4*p1*p3 + Y^3*p3*q1)*q2*x^3 - (Y^4*p2*q3 + Y^3*p3)*x^2)*y - 1)

'A: ' [1 0]
[0 1]

### Example 76. Restricted Dyck Paths on Valleys Sequence

In [124]:
A = x^2*y+x^2-2*x*y-2*x+1
B = x^2*y^2+2*x^2*y-x*y^2-3*x*y+y
C = x^2*y^2-x*y^2
f = (-B + sqrt(B^2-4*A*C))/(2*A)
P = Y^2*x^2*y+Y*x^2*y^2+Y^2*x^2-2*Y^2*x*y+2*Y*x^2*y-Y*x*y^2+x^2*y^2-2*Y^2*x-3*Y*x*y-x*y^2+Y^2+Y*y

def terms(n): return f.taylor([x, y], 0, n)

In [125]:
Dt = (x^3*y^2 - x^2*y^2 + 2*x^2*y + 2*x*y + x - 1)*(x - 1)
Q = (Y-1)^2 - Dt
W = (-B*(1-Y)+y*Dt)/(2*A*(1-Y))

In [126]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)
show("Safonov Rational Embedding: ", ratSaf)
show("A: ", A)

'Denef & Lipshitz Embedding: ' (Y^4*x^3*y^2 - Y^3*x^2*y^2 + 2*Y^2*x^2*y + Y^2*x*y + Y*x*y + 2*Y*x - x - 1)*(Y*x - 1)*Y^2*y/((Y^5*x^4*y^2 - 2*Y^4*x^3*y^2 + 2*Y^3*x^3*y + Y^3*x^2*y^2 + Y*x^2 - 2*Y*x*y - Y - 2*x + 2)*(Y^3*x^2*y + Y^2*x^2 - 2*Y^2*x*y - 2*Y*x + 1))

'Safonov Rational Embedding: ' ((2*Y^8 + Y^7)*x^2*y^4 - (4*Y^5 + 3*Y^4)*x*y^2 + (2*(Y^7 + Y^6)*x^2 - (4*Y^6 + Y^5)*x)*y^3 + (2*Y^3 + Y^2)*y)/((Y^6 + Y^5)*x^2*y^3 - (2*Y^3 + 3*Y^2 + Y)*x*y + ((Y^5 + 2*Y^4 + Y^3)*x^2 - (2*Y^4 + Y^3)*x)*y^2 + Y + 1)

'A: ' [1 0]
[1 1]

### Example 77. Zeilberger's Binary Tree Jump Statistics

In [127]:
f = - (-z*y*x+y*x+sqrt(z^2*y^2*x^2 -2*z*y^2*x^2-2*z*y^2*x+y^2*x^2-2*y^2*x+y^2) +y-2)/(2*(z*y*x+y^2*x-y*x-y+1))-1
P = ((Y+1)^2*x*y^2 + (Y+1)^2*x*y*z - (Y+1)^2*x*y - (Y+1)*x*y*z - (Y+1)^2*y + (Y+1)*x*y + (Y+1)^2 + (Y+1)*y - 2*(Y+1) + 1)*(x*y^2 + x*y*z - x*y - y + 1)

def terms(n): return f.taylor([x, y, z], 0, n)

In [128]:
C = (x*y^2 + x*y*z - x*y - y + 1)*x*y^2
B = (2*x*y^2 + x*y*z - x*y - y)*(x*y^2 + x*y*z - x*y - y + 1)
A = (x*y^2 + x*y*z - x*y - y + 1)^2

Dt = x^2*z^2 - 2*x^2*z + x^2 - 2*x*z - 2*x + 1
Q = (Y-1)^2 - Dt
W = (-B*(1-Y)+((x*y^2 + x*y*z - x*y - y + 1)*y)*Dt)/(2*A*(1-Y))

In [129]:
ratDL = embed_algebraic_DL(P, Q=Q, W=W, Y=Y)
ratSaf, A = embed_algebraic_Safonov(P, terms, Y=Y)
show("Denef & Lipshitz Embedding: ", ratDL)
show("Safonov Rational Embedding: ", ratSaf)
show("A: ", A)

'Denef & Lipshitz Embedding: ' (Y^4*x^2*z^2 - 2*Y^3*x^2*z + 2*Y^3*x*y + Y^3*x*z + Y^2*x^2 - 2*Y^2*x*y - 3*Y^2*x*z - Y^2*x - Y*x - Y + 2)*Y*y/((Y^3*x^2*z^2 - 2*Y^2*x^2*z + Y*x^2 - 2*Y*x*z - Y - 2*x + 2)*(Y^3*x*y^2 + Y^3*x*y*z - Y^2*x*y - Y*y + 1))

'Safonov Rational Embedding: ' (2*Y^7*x*y^4 - 2*(Y^6 - Y^5)*x*y^3 - (Y^4*x + 2*Y^4)*y^2 + (2*Y^3 - Y^2)*y + (2*Y^8*x*y^4 + Y^6*x*y^3)*z)/(Y^5*x*y^3 - (Y^4 - 2*Y^3)*x*y^2 - (Y^2 + (Y^2 - Y)*x)*y + (Y^6*x*y^3 + Y^4*x*y^2)*z + Y - 1)

'A: ' [1 0 0]
[1 1 1]
[0 0 1]